<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC1_Turbofan_FD002_Model_Distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

In [2]:
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.exceptions import ConvergenceWarning

from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [3]:
sns.set_palette('colorblind')

# Data Preparation

In [4]:
# Dataset Download 
os.system('git clone https://github.com/arthursl12/dataset_2')
os.system('mv /content/dataset_2/CMaps /content/CMaps')
os.system('mv /content/dataset_2/data_processing /content/data_processing')
os.system('rm -rf dataset_2')

0

In [5]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [6]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [7]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(2)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260,312,20.0037,0.7000,100.0,491.19,608.79,1495.60,1269.51,9.35,...,314.05,2389.02,8169.64,9.3035,0.03,369,2324,100.00,24.36,14.5189
53755,260,313,10.0022,0.2510,100.0,489.05,605.81,1514.32,1324.12,10.52,...,371.22,2388.42,8245.36,8.7586,0.03,374,2319,100.00,28.10,16.9454
53756,260,314,25.0041,0.6200,60.0,462.54,537.48,1276.24,1057.92,7.05,...,163.74,2030.33,7971.25,11.0657,0.02,310,1915,84.93,14.19,8.5503
53757,260,315,25.0033,0.6220,60.0,462.54,537.84,1272.95,1066.30,7.05,...,164.37,2030.35,7972.47,11.0537,0.02,311,1915,84.93,14.05,8.3729


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [8]:
test.shape, y_test.shape

((33991, 26), (259, 1))

In [9]:
test_last = proc.transform_test_keep_setting(test)
test_last.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,10.0076,0.2501,100.0,489.05,605.42,1515.00,1325.07,10.52,15.50,393.58,...,370.87,2388.32,8167.06,8.7456,0.03,371,2319,100.0,28.30,17.0934
1,0.0018,0.0000,100.0,518.67,642.67,1591.67,1418.17,14.62,21.61,553.36,...,521.10,2388.12,8138.12,8.4248,0.03,393,2388,100.0,38.82,23.3463
2,35.0015,0.8412,100.0,449.44,555.86,1370.62,1135.59,5.48,8.00,194.58,...,183.11,2388.07,8071.23,9.3094,0.02,332,2223,100.0,14.75,8.9589
3,20.0032,0.7000,100.0,491.19,607.99,1487.94,1257.49,9.35,13.66,334.39,...,314.88,2388.12,8062.39,9.2349,0.02,365,2324,100.0,24.22,14.6814
4,42.0055,0.8400,100.0,445.00,550.81,1358.95,1140.34,3.91,5.72,138.42,...,130.82,2389.06,8140.94,9.3964,0.02,333,2212,100.0,10.34,6.3601


In [10]:
X_test = test_last

### Remaining Useful Life (RUL)

In [11]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,148
1,1,2,147
2,1,3,146
3,1,4,145
4,1,5,144


## Attributes and target separation

In [12]:
X_train, y_train = proc.X_y_train_divide_with_settings(train)

In [13]:
y_train.head()

,RUL
0,148
1,147
2,146
3,145
4,144


In [14]:
X_train.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,9.02,175.71,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,9.03,175.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


## Training and Evaluation functions

In [15]:
eval = Evaluation()

In [16]:
search = HyperparameterSearch()

# DummyRegressor

Just to have a baseline on models. The trained models must have a better performance than a dummy.

## Linear RUL

In [17]:
model = Pipeline([
    ('dummy_reg'  ,   DummyRegressor())
])

In [18]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.000,RMSE=-69.180
(CV) R2=-0.003,RMSE=-69.154


In [19]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.251,RMSE=-60.162


## Non-linear RUL

In [20]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DummyRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':90})))
    ])

In [21]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1)
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 71
n_resources: 500
Fitting 5 folds for each of 71 candidates, totalling 355 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 71 is smaller than n_iter=107. Running 71 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


----------
iter: 1
n_candidates: 24
n_resources: 1500
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 2
n_candidates: 8
n_resources: 4500
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 3
n_candidates: 3
n_resources: 13500
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 119}}
Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DummyRegressor(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f8569b4bc20>,
                                                                            kw_args={'a_max': 119,
                                                                                     'a_min': 0})))])


In [22]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DummyRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':119})))
    ])

In [23]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.000,RMSE=-39.479
(CV) R2=-0.001,RMSE=-39.484


In [24]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(y_test, model.predict(X_test))

R2=-0.004,RMSE=-53.879


## PolyFeatures + Linear RUL

In [25]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('tree_reg'  ,   DummyRegressor())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('tree_reg', DummyRegressor())])

In [26]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 12
n_resources: 500
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 12 is smaller than n_iter=107. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


----------
iter: 1
n_candidates: 4
n_resources: 1500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 2
n_candidates: 2
n_resources: 4500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'poly_ft__interaction_only': False, 'poly_ft__include_bias': False, 'poly_ft__degree': 3}
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('tree_reg', DummyRegressor())])


In [27]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('tree_reg'  ,   DummyRegressor())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('tree_reg', DummyRegressor())])

In [28]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.000,RMSE=-69.180
(CV) R2=-0.003,RMSE=-69.154


In [29]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.251,RMSE=-60.162


## PolyFeatures + Non-Linear RUL

In [30]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DummyRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':90})))
])

In [31]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1)
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 119}, 'poly_ft__interaction_only': True, 'poly_ft__include_bias': False, 'poly_ft__degree': 1}
Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=1, include_bias=False,
                 

In [32]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=1, include_bias=False,
                                    interaction_only=True)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DummyRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':119})))
])

In [33]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.000,RMSE=-39.479
(CV) R2=-0.001,RMSE=-39.484


In [34]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(y_test, model.predict(X_test))

R2=-0.004,RMSE=-53.879


# KNeighborsRegressor

## Linear RUL

In [ ]:
model = Pipeline([
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('neigh_reg'  ,   KNeighborsRegressor())
])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "neigh_reg__n_neighbors": [3,5,15,25,45,55,65,75,85,95],
        "neigh_reg__weights": ["uniform", "distance"],
        "neigh_reg__algorithm": ["auto", "ball_tree", "kd_tree"],
        "neigh_reg__leaf_size": [20,30,50,100,150],
        "neigh_reg__p": [2,5,10],
        "neigh_reg__metric": ["minkowski", "euclidean", "manhattan", "chebyshev"],
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'neigh_reg__weights': 'distance', 'neigh_reg__p': 2, 'neigh_reg__n_neighbors': 25, 'neigh_reg__metric': 'manhattan', 'neigh_reg__leaf_size': 50, 'neigh_reg__algorithm': 'kd_tree'}
Pipeline(steps=[('selection',
                 SelectFromModel(estimator=Decision

Best Model:
```
KNeighborsRegressor(algorithm='kd_tree', leaf_size=50,
                    metric='manhattan', n_neighbors=25,
                    weights='distance'))
```

In [ ]:
model = Pipeline([
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('neigh_reg'  ,   KNeighborsRegressor(algorithm='kd_tree', leaf_size=50,
                                     metric='manhattan', n_neighbors=25,
                                     weights='distance'))
])
model

Pipeline(steps=[('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('neigh_reg',
                 KNeighborsRegressor(algorithm='kd_tree', leaf_size=50,
                                     metric='manhattan', n_neighbors=25,
                                     weights='distance'))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=1.000,RMSE=-0.000
(CV) R2=0.491,RMSE=-49.263


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.474,RMSE=-39.010


It overfitted the training set. Poor results. Worse than linear regression.

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KNeighborsRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__n_neighbors": [3,5,15,25,45,55,65,75,85,95],
        "trf_reg__regressor__weights": ["uniform", "distance"],
        "trf_reg__regressor__algorithm": ["auto", "ball_tree", "kd_tree"],
        "trf_reg__regressor__leaf_size": [20,30,50,100,150],
        "trf_reg__regressor__p": [2,5,10],
        "trf_reg__regressor__metric": ["minkowski", "euclidean", "manhattan", "chebyshev"],
    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 89}, 'trf_reg__regressor__weights': 'distance', 'trf_reg__regressor__p': 2, 'trf_reg__regressor__n_neighbors': 15, 'trf_reg__regressor__metric': 'minkowski', 'trf_reg__regressor__leaf_size': 150, 'trf_reg__

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 89}}

KNeighborsRegressor(leaf_size=150,
                    n_neighbors=15,
                    weights='distance'),
```



In [ ]:
model = Pipeline([
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KNeighborsRegressor(leaf_size=150,
                                            n_neighbors=15,
                                            weights='distance'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':89})))
    ])
model

Pipeline(steps=[('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KNeighborsRegressor(leaf_size=150,
                                                                          n_neighbors=15,
                                                                          weights='distance'),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f41c8630c20>,
                                                                            kw_args={'a_max': 89,
                                                                                     'a_min': 0})))])

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=1.000,RMSE=-0.000
(CV) R2=0.700,RMSE=-15.308


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.742,RMSE=-15.957


Overfitted the training as well. Good generalization.

## PolyFeatures + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('neigh_reg'  ,   KNeighborsRegressor())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('neigh_reg', KNeighborsRegressor())])

In [ ]:
# ~40min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "neigh_reg__n_neighbors": [3,5,15,25,45,55,65,75,85,95],
        "neigh_reg__weights": ["uniform", "distance"],
        "neigh_reg__algorithm": ["auto", "ball_tree", "kd_tree"],
        "neigh_reg__leaf_size": [20,30,50,100,150],
        "neigh_reg__p": [2,5,10],
        "neigh_reg__metric": ["minkowski", "euclidean", "manhattan", "chebyshev"],
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'poly_ft__interaction_only': True, 'poly_ft__include_bias': True, 'poly_ft__degree': 3, 'neigh_reg__weights': 'uniform', 'neigh_reg__p': 5, 'neigh_reg__n_neighbors': 15, 'neigh_reg__metric': 'manhattan', 'neigh_reg__leaf_size': 50, 'neigh_reg__algorithm': 'ball

Best Model:
```
PolynomialFeatures(degree=3, interaction_only=True)),
KNeighborsRegressor(algorithm='ball_tree', leaf_size=50,
                                     metric='manhattan', n_neighbors=15,
                                     p=5))
```


In [ ]:
model = Pipeline([
    ('poly_ft' , PolynomialFeatures(degree=3, interaction_only=True)),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('neigh_reg' ,   KNeighborsRegressor(algorithm='ball_tree', leaf_size=50,
                                     metric='manhattan', n_neighbors=15,
                                     p=5))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, interaction_only=True)),
                ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('neigh_reg',
                 KNeighborsRegressor(algorithm='ball_tree', leaf_size=50,
                                     metric='manhattan', n_neighbors=15,
                                     p=5))])

In [ ]:
# ~5min
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.608,RMSE=-43.323
(CV) R2=0.541,RMSE=-46.862


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.592,RMSE=-34.347


Didn't improve much

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KNeighborsRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
# ~25min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__n_neighbors": [3,5,15,25,45,55,65,75,85,95],
        "trf_reg__regressor__weights": ["uniform", "distance"],
        "trf_reg__regressor__algorithm": ["auto", "ball_tree", "kd_tree"],
        "trf_reg__regressor__leaf_size": [20,30,50,100,150],
        "trf_reg__regressor__p": [2,5,10],
        "trf_reg__regressor__metric": ["minkowski", "euclidean", "manhattan", "chebyshev"],
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 53759
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 107
n_resources: 500
Fitting 5 folds for each of 107 candidates, totalling 535 fits
----------
iter: 1
n_candidates: 36
n_resources: 1500
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 2
n_candidates: 12
n_resources: 4500
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 13500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 77}, 'trf_reg__regressor__weights': 'uniform', 'trf_reg__regressor__p': 5, 'trf_reg__regressor__n_neighbors': 5, 'trf_reg__regressor__metric': 'euclidean', 'trf_reg__regressor__leaf_size': 100, 'trf_reg__re

Best Model:
```
'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 77}
PolynomialFeatures(include_bias=False, interaction_only=True)),
KNeighborsRegressor(algorithm='kd_tree',
                    leaf_size=100,
                    metric='euclidean',
                    p=5),
```


In [ ]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(include_bias=False, interaction_only=True)),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KNeighborsRegressor(algorithm='kd_tree',
                                            leaf_size=100,
                                            metric='euclidean',
                                            p=5),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':77})))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KNeighborsRegressor(algorithm='kd_tree',
                                                                          leaf_size=100,
                                                                          metric='euclidean',
                                                                          p=5),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f41c8630c20>,
                                                                            kw_args={'a_max': 77,
                                                                                     'a_min': 0})))])

In [ ]:
# ~5min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.831,RMSE=-9.536
(CV) R2=0.763,RMSE=-11.462


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.812,RMSE=-11.595


Didn't overfit. Good generalization.